# Resolving

Resolvers are helpers to find commonly used resources that one may want to link resources to. Resolvers are configured in the `Resolvers` section of the configuration file.

In [1]:
from kgforge.core import KnowledgeGraphForge

In [2]:
import getpass

In [3]:
token = getpass.getpass()

In [4]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", token=token)

# Imports

In [5]:
from kgforge.core.commons.strategies import ResolvingStrategy

## Discover scopes, resolvers and targets

With the `forge.resolvers()` method, the user can inspect the available scopes, resolvers, and targets.
A scope is linked to a data source (directory, store, or URL). Each scope will have at least one resolver, which is an implementation of the Forge Resolver. A resolver can have multiple targets, and it will delimitate the resolution within the resolver's scope.

In [6]:
forge.resolvers()

Available scopes:
 -  agent :
     - resolver:  AgentResolver
         - targets:  agents
 -  ontology :
     - resolver:  OntologyResolver
         - targets:  terms


## Ontology terms

To find resources using resolver, you can use regex expressions within the provided string.

### using BEST_MATCH strategy (default)

Use ontology terms to retrieve the "Primary somatosensory area, trunk, layer 1" using its notation "ssp-tr1":

In [7]:
brain_region = forge.resolve("ssp-tr1", scope="ontology", target="terms")

In [8]:
type(brain_region)

kgforge.core.resource.Resource

In [9]:
print(brain_region)

{
    id: http://api.brain-map.org/api/v2/data/Structure/1006
    type: Class
    label: Primary somatosensory area, trunk, layer 1
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: SSp-tr1
    prefLabel: Primary somatosensory area, trunk, layer 1
    subClassOf: BrainRegion
}


If there is a unique scope, resolver, or target, it is possible to omit them in the parametrization.

In [10]:
result = forge.resolve("ssp-tr1", scope="ontology")

In [11]:
print(result)

{
    id: http://api.brain-map.org/api/v2/data/Structure/1006
    type: Class
    label: Primary somatosensory area, trunk, layer 1
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: SSp-tr1
    prefLabel: Primary somatosensory area, trunk, layer 1
    subClassOf: BrainRegion
}


In [12]:
brain_region == result

True

### using ALL_MATCHES strategy

Use ontology terms to retreieve a "somatosensory" followed by "trunk" brain region:

In [13]:
brain_regions = forge.resolve("somatosensory.*trunk", scope="ontology", strategy=ResolvingStrategy.ALL_MATCHES, limit=3)

In [14]:
for x in brain_regions:
    print(x)

{
    id: http://api.brain-map.org/api/v2/data/Structure/1086
    type: Class
    label: Primary somatosensory area, trunk, layer 4
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: SSp-tr4
    prefLabel: Primary somatosensory area, trunk, layer 4
    subClassOf: BrainRegion
}
{
    id: http://api.brain-map.org/api/v2/data/Structure/1111
    type: Class
    label: Primary somatosensory area, trunk, layer 5
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: SSp-tr5
    prefLabel: Primary somatosensory area, trunk, layer 5
    subClassOf: BrainRegion
}
{
    id: http://api.brain-map.org/api/v2/data/Structure/1006
    type: Class
    label: Primary somatosensory area, trunk, layer 1
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: SSp-tr1
    prefLabel: Primary somatosensory area, trunk, layer 1
    subClassOf: BrainRegion
}


### using EXACT_MATCH strategy

For this strategy, the exact string has to be provided, and it is case sensitive.

In [15]:
result = forge.resolve("Somatosensory areas, layer 1", scope="ontology", strategy=ResolvingStrategy.EXACT_MATCH)

In [16]:
print(result)

{
    id: http://api.brain-map.org/api/v2/data/Structure/12993
    type: Class
    label: Somatosensory areas, layer 1
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: SS1
    prefLabel: Somatosensory areas, layer 1
    subClassOf: BrainRegion
}


## Agent Entities

### scope
Resolving an entity resource having "anne".

In [17]:
agent = forge.resolve("anne", scope="agent")

In [18]:
type(agent)

kgforge.core.resource.Resource

In [19]:
print(agent)

{
    id: https://www.grid.ac/institutes/grid.5333.6
    type:
    [
        Organization
        Agent
    ]
    name: École Polytechnique Fédérale de Lausanne
}


### using ALL_MATCHES strategy

In [20]:
agents = forge.resolve("anne", scope="agent", strategy=ResolvingStrategy.ALL_MATCHES)

In [21]:
for x in agents:
    print(x)

{
    id: https://staging.epfl.ch/nexus/v1/test/bbp/users/jbeaumon
    type:
    [
        Agent
        Person
    ]
    familyName: Beaumon
    givenName: Joanne
}
{
    id: https://www.grid.ac/institutes/grid.5333.6
    type:
    [
        Agent
        Organization
    ]
    name: École Polytechnique Fédérale de Lausanne
}


### using EXACT_MATCH strategy

In [22]:
agent = forge.resolve("anne", scope="agent", strategy=ResolvingStrategy.EXACT_MATCH)

In [23]:
print(agent)

None


## filtering by type

In [24]:
person = forge.resolve("anne", scope="agent", type="Person")

In [25]:
print(person)

{
    id: https://staging.epfl.ch/nexus/v1/test/bbp/users/jbeaumon
    type:
    [
        Agent
        Person
    ]
    familyName: Beaumon
    givenName: Joanne
}


In [26]:
organization = forge.resolve("anne", scope="agent", type="Organization")

In [27]:
print(organization)

{
    id: https://www.grid.ac/institutes/grid.5333.6
    type:
    [
        Organization
        Agent
    ]
    name: École Polytechnique Fédérale de Lausanne
}
